In [1]:
12 * 34

408

In [2]:
(2 + 1*10) * (4 + 3*10)

408

In [3]:
from sympy import poly
from sympy.abc import x

poly(2 + 1*x) * poly(4 + 3*x)

Poly(3*x**2 + 10*x + 8, x, domain='ZZ')

In [4]:
# a and b are lists of polynomial coefficients, with a[i] being the coefficient of x**i.
def schoolbook_mult_2x2_poly(a, b):
    c = [0] * 3
    c[0] = a[0] * b[0]
    c[1] = a[1] * b[0] + a[0] * b[1]
    c[2] = a[1] * b[1]
    return c

In [5]:
schoolbook_mult_2x2_poly([2, 1], [4, 3])

[8, 10, 3]

In [6]:
def karatsuba_mult_poly(a, b):
    c = [0] * 3
    c[0] = a[0] * b[0]
    c[2] = a[1] * b[1]
    t = (a[0] + a[1]) * (b[0] + b[1])
    c[1] = t - c[0] - c[2]
    return c

In [7]:
karatsuba_mult_poly([2, 1], [4, 3])

[8, 10, 3]

In [8]:
def karatsuba_mult_poly_alt(a, b):
    c = [0] * 3
    c[0] = a[0] * b[0]
    c[2] = a[1] * b[1]
    s_a = +1 if a[0] > a[1] else -1
    s_b = +1 if b[0] > b[1] else -1
    t = abs(a[0] - a[1]) * abs(b[0] - b[1])
    c[1] = c[0] + c[2] - s_a * s_b * t
    return c

In [9]:
karatsuba_mult_poly_alt([2, 1], [4, 3])

[8, 10, 3]

In [10]:
123 * 456

56088

In [11]:
def schoolbook_mult3x3_poly(a, b):
    c = [0] * 5
    c[0] = a[0] * b[0]
    c[1] = a[1] * b[0] + a[0] * b[1]
    c[2] = a[2] * b[0] + a[1] * b[1] + a[0] * b[2]
    c[3] = a[2] * b[1] + a[1] * b[2]
    c[4] = a[2] * b[2]
    return c

In [12]:
schoolbook_mult3x3_poly([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

In [13]:
from sympy import solve
from sympy.matrices import Matrix

def poly_from_coeffs(coeffs):
    return poly(sum(c * (x**i) for i, c in enumerate(coeffs)))

def lagrange_interpolate_deg4(points):
    (x0, y0), (x1, y1), (x2, y2), (x3, y3), (x4, y4) = points
    A = Matrix([
        [x0**0, x0**1, x0**2, x0**3, x0**4],
        [x1**0, x1**1, x1**2, x1**3, x1**4],
        [x2**0, x2**1, x2**2, x2**3, x2**4],
        [x3**0, x3**1, x3**2, x3**3, x3**4],
        [x4**0, x4**1, x4**2, x4**3, x4**4],
    ])
    b = Matrix([y0, y1, y2, y3, y4])
    coeffs = A.solve(b)
    return poly_from_coeffs(coeffs)

In [14]:
lagrange_interpolate_deg4([(0, 5), (1, 6), (2, 21), (3, 86), (4, 261)])

Poly(x**4 + 5, x, domain='ZZ')

In [15]:
lagrange_interpolate_deg4([(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)])

Poly(x, x, domain='ZZ')

In [16]:
def toom_cook_mult3x3_poly_v0(a, b):
    xs = [-2, -1, 0, 1, 2]
    axs = [poly_from_coeffs(a).eval(x) for x in xs]
    bxs = [poly_from_coeffs(b).eval(x) for x in xs]
    cxs = [ax * bx for ax, bx in zip(axs, bxs)]
    
    return lagrange_interpolate_deg4(zip(xs, cxs)).all_coeffs()

In [17]:
toom_cook_mult3x3_poly_v0([3, 2, 1], [6, 5, 4])

[4, 13, 28, 27, 18]

In [18]:
def lagrange_interpolate_coeffs_deg4_with_leading_term(points, c4):
    (x0, y0), (x1, y1), (x2, y2), (x3, y3) = points
    A = Matrix([
        [x0**0, x0**1, x0**2, x0**3, x0**4],
        [x1**0, x1**1, x1**2, x1**3, x1**4],
        [x2**0, x2**1, x2**2, x2**3, x2**4],
        [x3**0, x3**1, x3**2, x3**3, x3**4],
        [0, 0, 0, 0, 1],
    ])
    b = Matrix([y0, y1, y2, y3, c4])
    return A.solve(b)

def toom_cook_mult3x3_poly_v1(a, b):
    xs = [-1, 0, 1, 2]
    axs = [poly_from_coeffs(a).eval(x) for x in xs]
    bxs = [poly_from_coeffs(b).eval(x) for x in xs]
    cxs = [ax * bx for ax, bx in zip(axs, bxs)]
    c4 = a[2] * b[2]
    
    return lagrange_interpolate_coeffs_deg4_with_leading_term(zip(xs, cxs), c4)

In [19]:
toom_cook_mult3x3_poly_v1([3, 2, 1], [6, 5, 4])

Matrix([
[18],
[27],
[28],
[13],
[ 4]])

In [20]:
def toom_cook_mult3x3_poly_v2(a, b):
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    c4 = a[2] * b[2]
    
    return lagrange_interpolate_coeffs_deg4_with_leading_term([(-1, c_n1), (0, c_0), (1, c_1), (2, c_2)], c4)

In [21]:
toom_cook_mult3x3_poly_v2([3, 2, 1], [6, 5, 4])

Matrix([
[18],
[27],
[28],
[13],
[ 4]])

In [22]:
def lagrange_interpolate_coeffs_at_n1_0_1_2_with_leading_term_v0(y_n1, y0, y1, y2, c4):
    A = Matrix([
        [1, -1, 1, -1, 1],
        [1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [1, 2, 4, 8, 16],
        [0, 0, 0, 0, 1],
    ])
    b = Matrix([y_n1, y0, y1, y2, c4])
    return A.solve(b)

def toom_cook_mult3x3_poly_v3(a, b):
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    c4 = a[2] * b[2]
    
    return lagrange_interpolate_coeffs_at_n1_0_1_2_with_leading_term_v0(c_n1, c_0, c_1, c_2, c4)

In [23]:
toom_cook_mult3x3_poly_v3([3, 2, 1], [6, 5, 4])

Matrix([
[18],
[27],
[28],
[13],
[ 4]])

In [24]:
A = Matrix([
    [1, -1, 1, -1, 1],
    [1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1],
    [1, 2, 4, 8, 16],
    [0, 0, 0, 0, 1],
])
A.inv()

Matrix([
[   0,    1,    0,    0,  0],
[-1/3, -1/2,    1, -1/6,  2],
[ 1/2,   -1,  1/2,    0, -1],
[-1/6,  1/2, -1/2,  1/6, -2],
[   0,    0,    0,    0,  1]])

In [25]:
from sympy import Rational as R

def lagrange_interpolate_coeffs_at_n1_0_1_2_with_leading_term_v1(y_n1, y0, y1, y2, c4):
    A_inv = Matrix([
        [0, 1, 0, 0, 0],
        [R(-1, 3), R(-1, 2), 1, R(-1, 6), 2],
        [R(1, 2), -1, R(1, 2), 0, -1],
        [R(-1, 6), R(1, 2), R(-1, 2), R(1, 6), -2],
        [0, 0, 0, 0, 1],
    ])
    b = Matrix([y_n1, y0, y1, y2, c4])
    return A_inv * b

def toom_cook_mult3x3_poly_v4(a, b):
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    c4 = a[2] * b[2]
    
    return lagrange_interpolate_coeffs_at_n1_0_1_2_with_leading_term_v1(c_n1, c_0, c_1, c_2, c4)

In [26]:
toom_cook_mult3x3_poly_v4([3, 2, 1], [6, 5, 4])

Matrix([
[18],
[27],
[28],
[13],
[ 4]])

In [27]:
def toom_cook_mult3x3_poly_v5(a, b):
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    c4 = a[2] * b[2]
    
    A_inv = Matrix([
        [0, 1, 0, 0, 0],
        [R(-1, 3), R(-1, 2), 1, R(-1, 6), 2],
        [R(1, 2), -1, R(1, 2), 0, -1],
        [R(-1, 6), R(1, 2), R(-1, 2), R(1, 6), -2],
        [0, 0, 0, 0, 1],
    ])
    b = Matrix([c_n1, c_0, c_1, c_2, c4])
    return A_inv * b

toom_cook_mult3x3_poly_v5([3, 2, 1], [6, 5, 4])

Matrix([
[18],
[27],
[28],
[13],
[ 4]])

In [28]:
def toom_cook_mult3x3_poly_v6(a, b):
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    c4 = a[2] * b[2]
    
    c1 = R(-c_n1, 3) + R(-c_0, 2) + c_1 + R(-c_2, 6) + 2*c4
    c2 = R(c_n1, 2) - c_0 + R(c_1, 2) - c4
    c3 = R(-c_n1, 6) + R(c_0, 2) + R(-c_1, 2) + R(c_2, 6) - 2*c4
    return [c_0, c1, c2, c3, c4]

toom_cook_mult3x3_poly_v6([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

In [29]:
def toom_cook_mult3x3_poly_v7(a, b):
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    c4 = a[2] * b[2]
    
    c_0_half = R(c_0, 2)
    c_1_half = R(c_1, 2)
    t0 = R(c_2, 6) - 2*c4
    
    c1 = -c_0_half - R(c_n1, 3) + c_1 - t0
    c2 = R(c_n1, 2) - c_0 + c_1_half - c4
    c3 = c_0_half - R(c_n1, 6) - c_1_half + t0
    return [c_0, c1, c2, c3, c4]

toom_cook_mult3x3_poly_v7([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

In [30]:
def toom_cook_mult3x3_poly_v8(a, b):
    a_n1 = a[0] - a[1] + a[2]
    b_n1 = b[0] - b[1] + b[2]
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a[0] + a[1] + a[2]
    b_1 = b[0] + b[1] + b[2]
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    c4 = a[2] * b[2]
    
    t1 = R(3*c_0 + 2 * c_n1 + c_2, 6) - 2*c4
    t2 = R(c_1 + c_n1, 2)
    
    c1 = c_1 - t1
    c2 = t2 - c_0 - c4
    c3 = t1 - t2
    return [c_0, c1, c2, c3, c4]

toom_cook_mult3x3_poly_v8([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]

In [32]:
def toom_cook_mult3x3_poly_mca(a, b):
    a02 = a[0] + a[2]
    b02 = b[0] + b[2]
    a_n1 = a02 - a[1]
    b_n1 = b02 - b[1]
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a02 + a[1]
    b_1 = b02 + b[1]
    a_2 = a[0] + 2*a[1] + 4*a[2]
    b_2 = b[0] + 2*b[1] + 4*b[2]
    
    c_n1 = a_n1 * b_n1
    c_0 = a_0 * b_0
    c_1 = a_1 * b_1
    c_2 = a_2 * b_2
    
    c4 = a[2] * b[2]
    
    t1 = R(3*c_0 + 2*c_n1 + c_2, 6) - 2*c4
    t2 = R(c_1 + c_n1, 2)
    
    c1 = c_1 - t1
    c2 = t2 - c_0 - c4
    c3 = t1 - t2
    return [c_0, c1, c2, c3, c4]

toom_cook_mult3x3_poly_mca([3, 2, 1], [6, 5, 4])

[18, 27, 28, 13, 4]